In [1]:
"""
Code based on:
Shang et al "Edge Attention-based Multi-Relational Graph Convolutional Networks" -> https://github.com/Luckick/EAGCN
Coley et al "Convolutional Embedding of Attributed Molecular Graphs for Physical Property Prediction" -> https://github.com/connorcoley/conv_qsar_fast
"""
import os
import logging
import pickle

import numpy as np
import pandas as pd
import torch
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MolFromSmiles
from sklearn.metrics import pairwise_distances
from torch.utils.data import Dataset

use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
IntTensor = torch.cuda.IntTensor if use_cuda else torch.IntTensor
DoubleTensor = torch.cuda.DoubleTensor if use_cuda else torch.DoubleTensor


def load_data_from_smiles(smiles, add_dummy_node=True, one_hot_formal_charge=True):
    """Load and featurize data from lists of SMILES strings and labels.

    Args:
        x_smiles (list[str]): A list of SMILES strings.
        labels (list[float]): A list of the corresponding labels.
        add_dummy_node (bool): If True, a dummy node will be added to the molecular graph. Defaults to True.
        one_hot_formal_charge (bool): If True, formal charges on atoms are one-hot encoded. Defaults to False.

    Returns:
        A tuple (X, y) in which X is a list of graph descriptors (node features, adjacency matrices, distance matrices),
        and y is a list of the corresponding labels.
    """

    try:
        mol = MolFromSmiles(smiles)
        try:
            mol = Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol, maxAttempts=5000)
            AllChem.UFFOptimizeMolecule(mol)
            mol = Chem.RemoveHs(mol)
        except:
            AllChem.Compute2DCoords(mol)
        afm, adj, dist = featurize_mol(mol, add_dummy_node, one_hot_formal_charge)

    except ValueError as e:
        print('the SMILES ({}) can not be converted to a graph.\nREASON: {}'.format(smiles, e))

    return afm, adj, dist


def featurize_mol(mol, add_dummy_node, one_hot_formal_charge):
    """Featurize molecule.

    Args:
        mol (rdchem.Mol): An RDKit Mol object.
        add_dummy_node (bool): If True, a dummy node will be added to the molecular graph.
        one_hot_formal_charge (bool): If True, formal charges on atoms are one-hot encoded.

    Returns:
        A tuple of molecular graph descriptors (node features, adjacency matrix, distance matrix).
    """
    node_features = np.array([get_atom_features(atom, one_hot_formal_charge)
                              for atom in mol.GetAtoms()])
    #print(node_features.shape)
    adj_matrix = np.eye(mol.GetNumAtoms())
    for bond in mol.GetBonds():
        begin_atom = bond.GetBeginAtom().GetIdx()
        end_atom = bond.GetEndAtom().GetIdx()
        adj_matrix[begin_atom, end_atom] = adj_matrix[end_atom, begin_atom] = 1

    conf = mol.GetConformer()
    pos_matrix = np.array([[conf.GetAtomPosition(k).x, conf.GetAtomPosition(k).y, conf.GetAtomPosition(k).z]
                           for k in range(mol.GetNumAtoms())])
    dist_matrix = pairwise_distances(pos_matrix)

    if add_dummy_node:
        m = np.zeros((node_features.shape[0] + 1, node_features.shape[1] + 1))
        m[1:, 1:] = node_features
        m[0, 0] = 1.
        node_features = m

        m = np.zeros((adj_matrix.shape[0] + 1, adj_matrix.shape[1] + 1))
        m[1:, 1:] = adj_matrix
        adj_matrix = m

        m = np.full((dist_matrix.shape[0] + 1, dist_matrix.shape[1] + 1), 1e6)
        m[1:, 1:] = dist_matrix
        dist_matrix = m

    return node_features, adj_matrix, dist_matrix


def get_atom_features(atom, one_hot_formal_charge=True):
    """Calculate atom features.

    Args:
        atom (rdchem.Atom): An RDKit Atom object.
        one_hot_formal_charge (bool): If True, formal charges on atoms are one-hot encoded.

    Returns:
        A 1-dimensional array (ndarray) of atom features.
    """
    attributes = []

    attributes += one_hot_vector(
        atom.GetAtomicNum(),
        #[6, 8, 7, 16, 35, 19, 53, 11, 17, 9, 15, 46, 5, 14, 55, 1, 13, 3, 26, 24, 22, 12, 25, 29,
        # 44, 28, 78, 47, 30, 75, 56, 50, 27, 20, 34, 45, 58, 40, 80, 74, 70, 21, 999]
        [5, 6, 7, 8, 9, 15, 16, 17, 35, 53, 999]
        #[6, 7, 8, 9, 15, 16, 17, 35, 53]
        #[9, 6, 17, 7, 15, 8, 35, 53, 16]
    )

    attributes += one_hot_vector(
        len(atom.GetNeighbors()),
        [0, 1, 2, 3, 4, 5]
    )

    attributes += one_hot_vector(
        atom.GetTotalNumHs(),
        [0, 1, 2, 3, 4]
    )

    if one_hot_formal_charge:
        #print(one_hot_vector(atom.GetFormalCharge(),[-1, 0, 1]))
        #attributes += one_hot_vector(atom.GetFormalCharge(),[-1, 0, 1, 999])
        attributes += one_hot_vector(atom.GetFormalCharge(),[-1, 0, 1])
    else:
        attributes.append(atom.GetFormalCharge())

    attributes.append(atom.IsInRing())
    attributes.append(atom.GetIsAromatic())
    return np.array(attributes, dtype=np.float32)


def one_hot_vector(val, lst):
    """Converts a value to a one-hot vector based on options in lst"""
    if val not in lst:
        val = lst[-1]
    return map(lambda x: x == val, lst)


#dimension 29

/users/PCON0041/xiaohu/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('./BH_processed.csv')
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*')
r_dic = []
ri_list = []
ri_afm_list = []
ri_adj_list = []
ri_dist_list = []
reaction_index_list = []
molecule_index_list = []
ri_center_list = []
yield_list = []

for reaction_index in range(df.shape[0]):
    if reaction_index % 400 == 0:
        print(reaction_index)
    r = df['reactants_mapped'].values[reaction_index]
    r_center = df['core_index_atom'].values[reaction_index].split('>')
    for ri_index in range(len(r.split('.'))):
        ri = r.split('.')[ri_index]
        ri_afm, ri_adj, ri_dist = load_data_from_smiles(ri)
        #print(ri, ri_afm)
        #print(ri_afm.shape)
        ri_list.append(ri)
        ri_afm_list.append(ri_afm)
        ri_adj_list.append(ri_adj)
        ri_dist_list.append(ri_dist)
        ri_center_index = [0] + [int(k) + 1 for k in r_center[ri_index].split('.')] #dummy node also count as reaction center
        ri_center_list.append(np.array(ri_center_index))
        
        reaction_index_list.append(int(reaction_index))
        molecule_index_list.append(int(ri_index))
        yield_list.append(df['yield'].values[reaction_index])
    r_dic = {'reactants': ri_list, 'reactant_afm': ri_afm_list, 'reactant_adj': ri_adj_list, 'reactant_dist': ri_dist_list, 'reaction_index':reaction_index_list, 'molecule_index': molecule_index_list, 'center_index': ri_center_list, 'yield': yield_list}
     
print(len(ri_list))
with open('BH_reactants_feats.pkl', 'wb') as f:
    pickle.dump(r_dic, f)

0
400
800
1200
1600
2000
2400
2800
3200
3600
7910


In [4]:
df = pd.read_csv('./BH_processed.csv')
r_dic = []
ri_list = []
ri_afm_list = []
ri_adj_list = []
ri_dist_list = []
reaction_index_list = []
molecule_index_list = []
yield_list = []

for reaction_index in range(df.shape[0]):
    if reaction_index % 400 == 0:
        print(reaction_index)
    r = df['reagents'].values[reaction_index]
    for ri_index in range(len(r.split('.'))):
        ri = r.split('.')[ri_index]
        ri_afm, ri_adj, ri_dist = load_data_from_smiles(ri)
        ri_list.append(ri)
        ri_afm_list.append(ri_afm)
        ri_adj_list.append(ri_adj)
        ri_dist_list.append(ri_dist)
        
        reaction_index_list.append(int(reaction_index))
        molecule_index_list.append(int(ri_index))
        yield_list.append(df['yield'].values[reaction_index])
    r_dic = {'reagents': ri_list, 'reagent_afm': ri_afm_list, 'reagent_adj': ri_adj_list, 'reagent_dist': ri_dist_list, 'reaction_index':reaction_index_list, 'molecule_index': molecule_index_list, 'yield': yield_list}
print(len(ri_list))  
with open('BH_reagents_feats.pkl', 'wb') as f:
    pickle.dump(r_dic, f)

0
400
800
1200
1600
2000
2400
2800
3200
3600
11865


In [5]:
df = pd.read_csv('./BH_processed.csv')
r_dic = []
ri_list = []
ri_afm_list = []
ri_adj_list = []
ri_dist_list = []
reaction_index_list = []
molecule_index_list = []
yield_list = []

for reaction_index in range(df.shape[0]):
    if reaction_index % 1000 == 0:
        print(reaction_index)
    r = df['products'].values[reaction_index]
    for ri_index in range(len(r.split('.'))):
        ri = r.split('.')[ri_index]
        ri_afm, ri_adj, ri_dist = load_data_from_smiles(ri)
        ri_list.append(ri)
        ri_afm_list.append(ri_afm)
        ri_adj_list.append(ri_adj)
        ri_dist_list.append(ri_dist)
        
        reaction_index_list.append(int(reaction_index))
        molecule_index_list.append(int(ri_index))
        yield_list.append(df['yield'].values[reaction_index])
    r_dic = {'products': ri_list, 'product_afm': ri_afm_list, 'product_adj': ri_adj_list, 'product_dist': ri_dist_list, 'reaction_index':reaction_index_list, 'molecule_index': molecule_index_list, 'yield': yield_list}
print(len(ri_list))    
with open('BH_products_feats.pkl', 'wb') as f:
    pickle.dump(r_dic, f)

0
1000
2000
3000
3955


In [6]:
ri_afm_list[0].shape

(19, 29)

In [3]:
def pad_array(array, shape, dtype=np.float32):
    """Pad a 2-dimensional array with zeros.

    Args:
        array (ndarray): A 2-dimensional array to be padded.
        shape (tuple[int]): The desired shape of the padded array.
        dtype (data-type): The desired data-type for the array.

    Returns:
        A 2-dimensional array of the given shape padded with zeros.
    """
    padded_array = np.zeros(shape, dtype=dtype)
    padded_array[:array.shape[0], :array.shape[1]] = array
    return padded_array


def mol_collate_func(batch_adjacency_matrix, batch_node_matrix, batch_distance_matrix):
    """Create a padded batch of molecule features.

    Args:
        batch (list[Molecule]): A batch of raw molecules.

    Returns:
        A list of FloatTensors with padded molecule features:
        adjacency matrices, node features, distance matrices, and labels.
    """
    adjacency_list, distance_list, features_list = [], [], []

    max_size = 0
    for i in range(batch_adjacency_matrix.shape[0]):
        molecule_adjacency_matrix = batch_adjacency_matrix[i]
        molecule_node_matrix = batch_node_matrix[i]
        molecule_distance_matrix = batch_distance_matrix[i]
        
        if molecule_adjacency_matrix.shape[0] > max_size:
            max_size = molecule_adjacency_matrix.shape[0]
            
    #print(max_size)
    for i in range(batch_adjacency_matrix.shape[0]):
        adjacency_list.append(pad_array(batch_adjacency_matrix[i], (max_size, max_size)))
        distance_list.append(pad_array(batch_distance_matrix[i], (max_size, max_size)))
        features_list.append(pad_array(batch_node_matrix[i], (max_size, batch_node_matrix[i].shape[1])))

    return [FloatTensor(features) for features in (adjacency_list, features_list, distance_list)]

In [4]:
"""
Dataloader
"""

import torch
import pickle
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import numpy as np

class Graph_DataLoader(Dataset):
    def __init__(self, data, data_dir):
        self.data_dir = data_dir
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return idx, self.data[idx,5].__float__()/100 # reactants, yield
        
        

In [5]:
import torch
import os
os.chdir('src')
from transformer import make_model

num_layers_to_keep = 6
model_params = {
    'd_atom': 28,
    'd_model': 1024,
    'N': num_layers_to_keep,
    'h': 16,
    'N_dense': 1,
    'lambda_attention': 0.33, 
    'lambda_distance': 0.33,
    'leaky_relu_slope': 0.1, 
    'dense_output_nonlinearity': 'relu', 
    'distance_matrix_kernel': 'exp', 
    'dropout': 0.0,
    'aggregation_type': 'mean'
}

pretrained_name = '../pretrained_weights.pt'  # This file should be downloaded first (See README.md).
pretrained_state_dict = torch.load(pretrained_name)

# Define the layers to keep
layers_to_keep = [
    "src_embed.lut.weight",  # these are additional the layers to keep
    "src_embed.lut.bias",
    "generator.proj.weight",
    "generator.proj.bias",
    *[f"encoder.layer.{i}." for i in range(num_layers_to_keep)]  # Keep the first 6 self-attention layers
]
model = make_model(**model_params)
model_state_dict = model.state_dict()

# Filter out the parameters corresponding to the specified layers
filtered_state_dict = {k: v for k, v in pretrained_state_dict.items() if any(layer in k for layer in layers_to_keep)}
for name, param in filtered_state_dict.items():
    if 'generator' in name:
         continue
    if isinstance(param, torch.nn.Parameter):
        param = param.data
    model_state_dict[name].copy_(param)
print('Updated model parameter name')
for name, param_i in model.named_parameters():
    print(name, param_i.size)

Updated model parameter name
encoder.layers.0.self_attn.linears.0.weight <built-in method size of Parameter object at 0x2b77c95a7818>
encoder.layers.0.self_attn.linears.0.bias <built-in method size of Parameter object at 0x2b77c95a7868>
encoder.layers.0.self_attn.linears.1.weight <built-in method size of Parameter object at 0x2b77c95a78b8>
encoder.layers.0.self_attn.linears.1.bias <built-in method size of Parameter object at 0x2b77c95a7908>
encoder.layers.0.self_attn.linears.2.weight <built-in method size of Parameter object at 0x2b77c95a7958>
encoder.layers.0.self_attn.linears.2.bias <built-in method size of Parameter object at 0x2b77c95a79a8>
encoder.layers.0.self_attn.linears.3.weight <built-in method size of Parameter object at 0x2b77c95a79f8>
encoder.layers.0.self_attn.linears.3.bias <built-in method size of Parameter object at 0x2b77c95a7a48>
encoder.layers.0.feed_forward.linears.0.weight <built-in method size of Parameter object at 0x2b77c95a7a98>
encoder.layers.0.feed_forward.l

In [6]:
import numpy as np
import torch 
from torch.utils.data import DataLoader
"""
Dataloader
"""

import torch
import pickle
from torch.utils.data import Dataset
import numpy as np

class Graph_DataLoader(Dataset):
    def __init__(self, data, data_dir):
        self.data_dir = data_dir
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return idx, self.data[idx,5].__float__()/100 #return reaction indices and yields
    
    
def get_reactant_batch(r_feats, set_index, batch_index):
    batch_index_indeed = set_index[batch_index] #the indices of batch_reactions in original dataset(3955)
    temp = torch.Tensor([])
    for i in range(len(batch_index)):
        temp = torch.cat((temp, torch.where(torch.Tensor(r_feats['reaction_index']) == batch_index_indeed[i])[0]), dim=0) # index is reactant index
    index = [int(i.item()) for i in temp] #indices of molecules in batch_reactions

    r_batch_adjacency_matrix = np.array(r_feats['reactant_adj'])[index] 
    r_batch_node_features = np.array(r_feats['reactant_afm'])[index]
    r_batch_distance_matrix = np.array(r_feats['reactant_dist'])[index]
    r_batch_center_index = np.array(r_feats['center_index'])[index]
    r_batch_index = np.array(r_feats['reaction_index'])[index]

    r_batch_adjacency_matrix, r_batch_node_features, r_batch_distance_matrix = mol_collate_func(r_batch_adjacency_matrix, r_batch_node_features, r_batch_distance_matrix) #padded function
    r_batch_mask = torch.sum(torch.abs(r_batch_node_features), dim=-1) != 0    

    return r_batch_adjacency_matrix, r_batch_node_features, r_batch_distance_matrix, r_batch_mask, r_batch_center_index, r_batch_index

In [8]:
import pickle 
import pandas as pd
df = pd.read_csv('../BH_processed.csv')

with open('../BH_reactant_feats.pkl', 'rb') as f:
    reactant_features = pickle.load(f)

data_dir = '/fs/ess/PCON0041/xiaohu/Yield_Predicion_M/Data/preprocessed_datasets/BH/BH_processed.csv'
with open('../train_test_idxs.pickle', 'rb') as handle:
    idx_dict = pickle.load(handle)
        
train_index = idx_dict['train_idx'][1]
test_index = idx_dict['test_idx'][1]
valid_index = train_index[int(0.9*len(train_index)):]
train_index = train_index[0:int(0.9*len(train_index))]

testset = Graph_DataLoader(df.loc[test_index].values, data_dir)
test_loader = DataLoader(testset, batch_size=32, shuffle=True)


In [9]:
for iter, (batch_index, batch_targets) in enumerate(test_loader): #batch_index is reaction index
    r_batch_adjacency_matrix, r_batch_node_features, r_batch_distance_matrix, r_batch_mask, r_batch_center_index, r_batch_index = get_reactant_batch(reactant_features, test_index, batch_index)

/users/PCON0041/xiaohu/anaconda3/envs/rxnfp/lib/python3.6/site-packages/ipykernel_launcher.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/users/PCON0041/xiaohu/anaconda3/envs/rxnfp/lib/python3.6/site-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/users/PCON0041/xiaohu/anaconda3/envs/rxnfp/lib/python3.6/site-packages/ipykernel_launcher.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprec

In [10]:



    #print(reactant_features['reaction_index'][test_index].shape)
    total_test_reaction_index = torch.Tensor(reactant_features['reaction_index'])[test_index]
    #print(total_test_features.shape)
    temp = torch.Tensor([])
    for i in range(len(batch_index)):
        temp = torch.cat((temp, torch.where(total_test_reaction_index == batch_index[i])[0]), dim=0) # index is reactant index
    index = [int(i.item()) for i in temp]

    batch_adjacency_matrix = np.array(reactant_features['reactant_adj'])[test_index][index] 
    batch_node_features = np.array(reactant_features['reactant_afm'])[test_index][index]
    batch_distance_matrix = np.array(reactant_features['reactant_dist'])[test_index][index]
    batch_center_index = np.array(reactant_features['center_index'])[test_index][index]
    print(batch_center_index)
    batch_adjacency_matrix, batch_node_features, batch_distance_matrix = mol_collate_func(batch_adjacency_matrix, batch_node_features, batch_distance_matrix)
    batch_mask = torch.sum(torch.abs(batch_node_features), dim=-1) != 0 
  
    print(batch_adjacency_matrix.shape, batch_node_features.shape, batch_distance_matrix.shape, batch_targets.shape, batch_center_index.shape)
    output = model(batch_node_features.to(device), batch_mask.to(device), batch_adjacency_matrix.to(device), batch_distance_matrix.to(device), None)
    print(output.shape)

[]
torch.Size([0]) torch.Size([0]) torch.Size([0]) torch.Size([3]) (0, 2)


/users/PCON0041/xiaohu/anaconda3/envs/rxnfp/lib/python3.6/site-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':
/users/PCON0041/xiaohu/anaconda3/envs/rxnfp/lib/python3.6/site-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.
/users/PCON0041/xiaohu/anaconda3/envs/rxnfp/lib/python3.6/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-o

NameError: name 'device' is not defined

In [10]:
mol = MolFromSmiles(smiles[0])
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol, maxAttempts=5000)
AllChem.UFFOptimizeMolecule(mol)
mol = Chem.RemoveHs(mol)
print(len(mol.GetAtoms()))

19


In [ ]:
element = [atom.GetSymbol() for atom in mol.GetAtoms()]
print(element)



['F', 'C', 'F', 'F', 'C', 'C', 'C', 'C', 'Cl', 'C', 'C', 'N', 'C', 'C', 'C', 'C', 'C', 'C', 'C']


In [ ]:
['F', 'C', 'F', 'F', 'C', 'C', 'C', 'C', 'Cl', 'C', 'C', 'N', 'C', 'C', 'C', 'C', 'C', 'C', 'C']

df_e = pd.DataFrame(x[0][1][1:,1:], index=element,  columns=element)
print(df_e)

      F    C    F    F    C    C    C    C   Cl    C    C    N    C    C    C  \
F   1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
C   1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
F   0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
F   0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
C   0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
C   0.0  0.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
C   0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
C   0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
Cl  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
C   0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0   
C   0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0   
N   0.0  0.0  0.0  0.0  0.0 

In [1]:
import torch

# Example tensors
tensor1 = torch.tensor([3, 1, 4, 1, 5])
tensor2 = torch.tensor([9, 2, 5, 8, 3])

# Sort tensor1 based on the values in tensor2
sorted_tensor1, indices = torch.sort(tensor1)
sorted_tensor2 = tensor2[indices]

print("Sorted tensor1:", sorted_tensor1)
print("Corresponding sorted tensor2:", sorted_tensor2)


/users/PCON0041/xiaohu/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sorted tensor1: tensor([1, 1, 3, 4, 5])
Corresponding sorted tensor2: tensor([2, 8, 9, 5, 3])


In [ ]:
tensor([ 367., 2061., 2829.,  583., 2242., 1751., 2143., 1497., 2631.,  700.,
        3310., 2754., 3321.,  603., 3361., 2932.,   47., 1383.,  821., 2349.,
        3717., 3277., 2084., 3177.,  334., 2552., 1090., 1972., 3162., 3326.,
        2628., 1992.])

tensor([  47.,  334.,  367.,  583.,  603.,  700.,  821., 1090., 1383., 1497.,
        1751., 1972., 1992., 2061., 2084., 2143., 2242., 2349., 2552., 2628.,
        2631., 2754., 2829., 2932., 3162., 3177., 3277., 3310., 3321., 3326.,
        3361., 3717.], device='cuda:0')